# Experiment MNIST - TSNE

This experiment tries to replicate the reult obtained by the paper https://arxiv.org/pdf/1906.00722.pdf where a process of dimensionality reduction was applied on the mnist dataset, and values of 0.946 for Trustworthiness and 0.938 for continuity were obtained. 

## Basic imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import numpy as np
import pandas as pd
from pathlib import Path
print(sys.path)
sys.path.append("../../")

['/home/hubert/librep-hiaac/experiments/Topological_ae', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages']


## Loading the dataset

In [3]:
from librep.transforms import TSNE
from librep.transforms import UMAP
from librep.datasets.multimodal import TransformMultiModalDataset, ArrayMultiModalDataset, WindowedTransform
from librep.metrics.dimred_evaluator import DimensionalityReductionQualityReport, MultiDimensionalityReductionQualityReport
from librep.datasets.har.loaders import MNISTView

2022-10-06 14:01:35.474507: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-06 14:01:35.474531: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
loader = MNISTView("../../data/old-views/MNIST/default/", download=False)
train_val_mnist, test_mnist = loader.load(concat_train_validation=True)

In [5]:
train_val_mnist, test_mnist

(PandasMultiModalDataset: samples=60000, features=785, no. window=1,
 PandasMultiModalDataset: samples=10000, features=785, no. window=1)

In [6]:
train_val_mnist.data

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
train_val_mnist.data.iloc[:,1:]

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# transform_umap = UMAP()
# umap_transform = WindowedTransform(
#    transform=UMAP(), fit_on="all", transform_on="all")

# transformer = TransformMultiModalDataset(transforms=[umap_transform])
# train_applied_umap = transformer(train_val_mnist)
# test_applied_umap = transformer(test_mnist)
# assert 1==0

In [9]:
# transform_tsne = TSNE()
# train_tsne = train_val_mnist.data.iloc[:,1:]
# print(train_val_mnist.shape)
# # assert 1==0

In [10]:
# metrics_reporter = DimensionalityReductionQualityReport(sampling_threshold=500)
# metrics_train_applied_tsne = metrics_reporter.evaluate([mnist_dataset_train, train_applied_tsne])

In [11]:
train_x = np.array(train_val_mnist.data.iloc[:,1:])
train_y = np.array(train_val_mnist.data.iloc[:,0])
test_x = np.array(test_mnist.data.iloc[:,1:])
test_y = np.array(test_mnist.data.iloc[:,0])

In [12]:
# dataset = tf.keras.datasets.mnist.load_data(path="mnist.npz")
# (train_x, train_y), (test_x, test_y) = dataset

In [13]:
# train_x_reordered = train_x.reshape((60000,-1))
# print('TRAIN_X_REORDERED', train_x_reordered.shape)

# test_x_reordered = test_x.reshape((10000,-1))
# print('TEST_X_REORDERED', test_x_reordered.shape)


In [14]:
mnist_dataset_train = ArrayMultiModalDataset(X=train_x, y=train_y, window_slices=[(0, 28*28)], 
                                             window_names=["px"])
mnist_dataset_test = ArrayMultiModalDataset(X=test_x, y=test_y, window_slices=[(0, 28*28)], 
                                             window_names=["px"])

In [ ]:
transform_tsne = TSNE()
transformer = TransformMultiModalDataset(transforms=[transform_tsne])
train_applied_tsne = transformer(mnist_dataset_train)
test_applied_tsne = transformer(mnist_dataset_test)

/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [ ]:
metrics_reporter = DimensionalityReductionQualityReport(sampling_threshold=60000)
metrics_train_applied_tsne = metrics_reporter.evaluate([mnist_dataset_train, train_applied_tsne])
print(metrics_train_applied_tsne)

In [ ]:
metrics_reporter = DimensionalityReductionQualityReport(sampling_threshold=10000)
metrics_test_applied_tsne = metrics_reporter.evaluate([mnist_dataset_test, test_applied_tsne])
print(metrics_test_applied_tsne)

In [ ]:
transform_umap = UMAP()
transformer = TransformMultiModalDataset(transforms=[transform_umap])
train_applied_umap = transformer(mnist_dataset_train)
test_applied_umap = transformer(mnist_dataset_test)

In [ ]:
metrics_reporter = DimensionalityReductionQualityReport(sampling_threshold=60000)
metrics_train_applied_umap = metrics_reporter.evaluate([mnist_dataset_train, train_applied_umap])
print(metrics_train_applied_umap)

In [ ]:
metrics_reporter = DimensionalityReductionQualityReport(sampling_threshold=10000)
metrics_test_applied_umap = metrics_reporter.evaluate([mnist_dataset_test, test_applied_umap])
print(metrics_test_applied_umap)